In [323]:
# could just get each artist from release.credits, then filter out non-musicians afterwards...

In [78]:
import discogs_client

token = 'eGkJSFluvBIeQCEmsueVEMZIabFbDLaPFIDcKFkg'
d = discogs_client.Client('ExampleApplication/0.1',user_token=token)

In [308]:
# d.search('Myron Walden', type='artist')[0].id
# d.artist(252331).name

In [309]:
# d = discogs_client.Client('ExampleApplication/0.1',user_token=token)
# a_id = d.search('Kurt Rosenwinkel', type='artist')[0].id
# releases = d.search(str(a_id), type='release')

In [4]:
import time
from collections import defaultdict

def discog(artist):
    # get the artist's id; search for releases
    d = discogs_client.Client('ExampleApplication/0.2',user_token=token)
    a_id = d.search(artist, type='artist')[0].id
    releases = d.search(str(a_id), type='release')
    r_dict = {}
    collab_dict = defaultdict(list)

    for i,release in enumerate(releases):
        r_title = release.title.split(' - ')[-1]
        time.sleep(1)
        #print r_title
        if release.master==None:
            continue
        elif release.master.id in r_dict:
            continue

        r_dict[release.master.id] = 1
        artist_ids = set([artist.id for artist in release.credits])
        #print artist_ids

        # add the album to list of collaborations
        for a_id in artist_ids:
            collab_dict[a_id] += [(r_title,release.year)]
        r_dict[r_title] = 1
    
    # sort each collaboration by year, then all collaborators by volume
    for artist in collab_dict.keys():
        collab_dict[artist] = sorted(collab_dict[artist],key=lambda a:a[1])

    collab_dict2 = []
    for key in sorted(collab_dict,key=lambda x:len(collab_dict[x]),reverse=True):
        collab_dict2.append((key,collab_dict[key]))
    return collab_dict2

In [5]:
artist = 'Myron Walden'
#artist_id = d.search(artist,type='artist')[0].id
a = discog(artist)
top_five = [collab for collab in a if collab[0] not in (artist_id,0)]
top_five = top_five[:5]

In [25]:
import numpy as np
import math
def pol2cart(rho, phi):
    x = rho * np.cos(phi)
    y = rho * np.sin(phi)
    return(x, y)

In [263]:
import networkx as nx

g = nx.MultiGraph()
g.add_node(0,name=artist,num=0)
ann_height = .025
pos = {0: (0,0)}
labels = {0: artist}
annots = [{'showarrow':False,'text':artist,'x':0,'xref':'x','y':ann_height,'yref':'y'}]
for i,collab in enumerate(top_five):
    a_name = d.artist(collab[0]).name
    g.add_node(i+1,name=a_name,num=i+1)
    labels[i+1] = a_name
    rho, phi = (1/float(len(collab[1]))), i*(2*math.pi/5)
    location = pol2cart(rho,phi)
    pos[i+1] = location
    a_dict = {'showarrow':False,'text':a_name,'x':location[0],'xref':'x','y':location[1]+ann_height,'yref':'y'}
    annots.append(a_dict)
    
    for album in collab[1]:
        g.add_edge(0,i+1)

label_pos = {}
for n, p in pos.iteritems():
    g.node[n]['pos'] = p
    label_pos[n] = map(sum,zip(p,(0,.05)))
    
# nx.draw(g,pos)
# __ = nx.draw_networkx_labels(g,label_pos,labels,nodelist=range(6))

In [305]:
from plotly.graph_objs import *

G = g
pos=nx.get_node_attributes(G,'pos')

edge_trace = Scatter(
    x=[],
    y=[],
    line=Line(width=0.5,color='#888'),
    hoverinfo='none',
    mode='lines')

for edge in G.edges():
    x0, y0 = G.node[edge[0]]['pos']
    x1, y1 = G.node[edge[1]]['pos']
    edge_trace['x'] += [x0, x1, None]
    edge_trace['y'] += [y0, y1, None]

node_trace = Scatter(
    x=[],
    y=[],
    text=[''],
    textposition='top',
    mode='markers',
    hoverinfo='text',
    marker=Marker(
        showscale=True,
        # colorscale options
        # 'Greys' | 'Greens' | 'Bluered' | 'Hot' | 'Picnic' | 'Portland' |
        # Jet' | 'RdBu' | 'Blackbody' | 'Earth' | 'Electric' | 'YIOrRd' | 'YIGnBu'
        colorscale='YIGnBu',
        reversescale=True,
        color=[],
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line=dict(width=2)))

for node in G.nodes():
    x, y = G.node[node]['pos']
    node_trace['x'].append(x)
    node_trace['y'].append(y)

node_trace['marker']['color'].append(0)
for i, node in enumerate(top_five):
    node_trace['marker']['color'].append(len(G.adj[i+1][0]))
    # add annotation if greater than 0
    node_info = '# of connections: '+str(len(G.adj[i+1][0]))
    node_info += '<br>' + '<br>'.join([str(tup[1])+': '+str(tup[0]) for tup in top_five[i][1]])
    node_trace['text'].append(node_info)

In [306]:
fig = Figure(data=Data([edge_trace, node_trace]),
             layout=Layout(
                title=artist + "'s Top Five Collaborators", #'<br>Network graph made with Python',
                titlefont=dict(size=16),
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations= annots,
                xaxis=XAxis(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=YAxis(showgrid=False, zeroline=False, showticklabels=False)))

py.iplot(fig, filename='networkx')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~jglub212/0 or inside your plot.ly account where it is named 'networkx'


In [307]:
node_trace['text']

['',
 '# of connections: 5<br>1998: Brian Blade Fellowship<br>2000: Perceptual<br>2003: Salt<br>2008: Season Of Changes<br>2014: Landmarks',
 '# of connections: 5<br>1998: Brian Blade Fellowship<br>2000: Perceptual<br>2003: Salt<br>2008: Season Of Changes<br>2014: Landmarks',
 '# of connections: 4<br>1998: Brian Blade Fellowship<br>2000: Perceptual<br>2008: Season Of Changes<br>2014: Landmarks',
 '# of connections: 3<br>1998: Brian Blade Fellowship<br>2000: Perceptual<br>2008: Season Of Changes',
 '# of connections: 2<br>2003: Salt<br>2003: Treats For The Nightwalker']

In [166]:
fig.data[1]

{'hoverinfo': 'text',
 'marker': {'color': [0, 0, 0, 0, 0, 0],
  'colorbar': {'thickness': 15,
   'title': 'Node Connections',
   'titleside': 'right',
   'xanchor': 'left'},
  'colorscale': 'YIGnBu',
  'line': {'width': 2},
  'reversescale': True,
  'showscale': True,
  'size': 10},
 'mode': 'markers',
 'text': ['# of connections: 0',
  '# of connections: 0',
  '# of connections: 0',
  '# of connections: 0',
  '# of connections: 0',
  '# of connections: 0'],
 'textposition': 'top',
 'type': 'scatter',
 'x': [0,
  0.20000000000000001,
  1.2246467991473533e-17,
  -0.25,
  -6.1232339957367648e-17,
  0.5],
 'y': [0,
  0.5,
  0.69999999999999996,
  0.5,
  0.16666666666666669,
  0.49999999999999989]}

In [161]:
for collab in top_five:
    print collab
    a_dict = {'showarrow':False,'text':}
    #fig.layout['annotations'].append(a_dict)

(252331, [(u'Brian Blade Fellowship', 1998), (u'Perceptual', 2000), (u'Salt', 2003), (u'Season Of Changes', 2008), (u'Landmarks', 2014)])
(510174, [(u'Brian Blade Fellowship', 1998), (u'Perceptual', 2000), (u'Salt', 2003), (u'Season Of Changes', 2008), (u'Landmarks', 2014)])
(510176, [(u'Brian Blade Fellowship', 1998), (u'Perceptual', 2000), (u'Season Of Changes', 2008), (u'Landmarks', 2014)])
(460771, [(u'Brian Blade Fellowship', 1998), (u'Perceptual', 2000), (u'Season Of Changes', 2008)])
(256512, [(u'Salt', 2003), (u'Treats For The Nightwalker', 2003)])


In [159]:
fig.layout['annotations'][0]

{'showarrow': False,
 'text': "Python code: <a href='https://plot.ly/ipython-notebooks/network-graphs/'> https://plot.ly/ipython-notebooks/network-graphs/</a>",
 'x': 0.005,
 'xref': 'paper',
 'y': -0.002,
 'yref': 'paper'}

In [124]:
f = Figure()

In [155]:
s = Scatter()
s?

In [156]:
s.anno

In [130]:
print collab[1]

[(u'Salt', 2003), (u'Treats For The Nightwalker', 2003)]


In [26]:
r = releases[10]
r.credits[0].name

u'Evelyn Morgan'

In [319]:
# many of the results don't have a 'master' attribute

In [320]:
#['Guitar','Piano','Drums','Synthesizer','Bass','Clarinet','Flute','Trumpet','Organ','Sax']

In [322]:
#discog('Myron Walden')

In [ ]:
t_d = {}
for a in r:
    title =  a.title.split(' - ')[-1]
    if title not in t_d:
        t_d[title] = 1
for key in t_d.keys():
    print key

In [3]:
results = d.search('Stockholm By Night', type='release')
results.pages

34

In [ ]:
def is_musician(s):
    instruments = ['Guitar','Piano','Drums','Synthesizer','Bass','Clarinet','Flute','Trumpet','Organ','Sax']
    return any(inst in s for inst in instruments)

In [ ]:
# artist = 'Myron Walden'
# d = discogs_client.Client('ExampleApplication/0.2',user_token=token)
# a_id = d.search(artist, type='artist')[0].id
# releases = d.search(str(a_id), type='release')
# r_dict = {}
# collab_dict = defaultdict(list)

# for i,release in enumerate(releases):
#     r_title = release.title.split(' - ')[-1]
#     time.sleep(1)
#     print r_title
#     if release.master==None:
#         continue
#     elif release.master.id in r_dict:
#         continue
    
#     r_dict[release.master.id] = 1
#     artist_ids = set([artist.id for artist in release.credits])
#     print artist_ids

#     # add the album to list of collaborations
#     for a_id in artist_ids:
#         collab_dict[a_id] += [(r_title,release.year)]
#     r_dict[r_title] = 1
    
#     if i==2:
#         break

# for artist in collab_dict.keys():
#     collab_dict[artist] = sorted(collab_dict[artist],key=lambda a:a[1])

# collab_dict2 = []
# for key in sorted(collab_dict,key=lambda x:len(collab_dict[x]),reverse=True):
#     collab_dict2.append((key,collab_dict[key]))

'''+ [ dict(
text="Python code: <a href='https://plot.ly/ipython-notebooks/network-graphs/'> https://plot.ly/ipython-notebooks/network-graphs/</a>",
showarrow=False,
xref="paper", yref="paper",
x=0.005, y=-0.002 ) ],'''